In [57]:
import numpy as np

In [58]:
dt = 1/30

A = np.array([[1, 0, dt, 0],
              [0, 1, 0, dt],
              [0, 0, 1, 0],
              [0, 0, 0, 1]], dtype=np.float32)
P_t = np.eye(A.shape[1], dtype=np.float32)
Q_t = np.eye(A.shape[1], dtype=np.float32)
E_t = np.array([[0], [0], [0], [0]], dtype=np.float32)


H_t = np.array([[1, 0, 0, 0],
                [0, 1, 0, 0]], dtype=np.float32)
R_t = np.eye(H_t.shape[0], dtype=np.float32)


In [59]:
def update_A(dt):
    global A
    A = np.array([[1, 0, dt, 0],
                [0, 1, 0, dt],
                [0, 0, 1, 0],
                [0, 0, 0, 1]], dtype=np.float32)

def predict():
    global E_t,P_t
    E_t = A @ E_t
    P_t = A @ P_t @ A.T + Q_t
    return E_t

def update(Z):
    global E_t,P_t
    K = P_t@H_t.T @ np.linalg.inv(H_t@P_t@H_t.T+R_t)
    E_t = E_t+K@(Z-H_t@E_t)
    P_t = (np.eye(H_t.shape[1], dtype=np.float32)-K@H_t)@P_t
    return E_t

In [60]:
from object_tracker_v1 import detect_obj
import cv2 as cv
import time


In [70]:

u = (160//2, 80*255//100, 80*255//100)
l = (90//2, 30*255//100, 30*255//100)
point=np.array([0,0],dtype=np.int32)

SOURCE='vid.mkv'
cap = cv.VideoCapture(SOURCE)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps= int(cap.get(cv.CAP_PROP_FPS))
print(fps)
out = cv.VideoWriter('out2.avi',cv.VideoWriter_fourcc('M','J','P','G'), fps, (frame_width,frame_height))
 
start = time.time()
while cap.isOpened():
    
    ret, frame = cap.read()
    if not ret:
        break
    point, r, _, c = detect_obj(frame, l, u)
    
    etat=predict()
    if point is not None:
        cv.circle(frame, point,
                  r, (0, 255, 255))
        cur_pt= np.int32(etat[:2].reshape((1,2))[0])
        next_pt=np.int32(etat[2:].reshape((1,2))[0])
        cv.circle(frame, cur_pt,
                  4, (255, 0, 255),5)
       
        cv.circle(frame, cur_pt+next_pt,
                  4, (0, 0, 255),5)


        cv.arrowedLine(frame,
                  cur_pt,cur_pt+next_pt,
                    color=(255,0, 0),
                    thickness=2,
                    tipLength=0.2)
    
        cv.circle(frame, point,
                  1, (255, 0, 0), 5)
        cv.polylines(frame, [c], True, (90, 90, 90))
        update(np.expand_dims(np.array([point[0],point[1]],dtype=np.float32),axis=-1))

    out.write(frame)
    frame2=cv.resize(frame,(frame.shape[1]//2,frame.shape[0]//2))
    cv.imshow('webcam', frame2)
    
    key = cv.waitKey(1) & 0xff

    end = time.time()
    if end!=start:
        dt=end - start
        update_A(end - start)
    else:
        time.sleep(0.1)
        update_A(0.1)
    start = time.time()
    if key == ord('q'):
        break


cv.destroyAllWindows()
cap.release()
out.release()


60
